# Install Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=de1b405f66eac2a4a793e084a6e9194ad5777ff79856ca6b28199cfe7e04e958
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


# Imports

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
import re
import json
import itertools
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from operator import add
import numpy as np
import nltk 
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords


from sklearn import metrics 

In [3]:
# Imports
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler, Imputer , StandardScaler
from pyspark.ml.classification import NaiveBayes

In [4]:
# Imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Download Data

In [ ]:
#!pip install --upgrade --no-cache-dir gdown

In [ ]:
#!gdown --id 1Ba72Gnrjre0KPqhWbQPbyYR3XBguosGf

In [ ]:
#!unzip 'imdb-review-dataset.zip' -d '/content/drive/MyDrive/BigData_FinalProject/Data'

In [ ]:
#!cp '/content/imdb-review-dataset.zip' '/content/drive/MyDrive/BigData_FinalProject'

In [ ]:
'''
! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/BigData_FinalProject/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
'''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#! kaggle datasets download ebiswas/imdb-review-dataset

100% 2.69G/2.69G [00:23<00:00, 132MB/s]
100% 2.69G/2.69G [00:23<00:00, 124MB/s]


In [ ]:
#!unzip imdb-review-dataset.zip

Archive:  imdb-review-dataset.zip
  inflating: part-01.json            
  inflating: part-02.json            
  inflating: part-03.json            
  inflating: part-04.json            
  inflating: part-05.json            
  inflating: part-06.json            
  inflating: sample.json             


In [ ]:
#!unzip 'imdb-review-dataset.zip' -d '/content/drive/MyDrive/BigData_FinalProject/Data'

In [ ]:
'''
f = open('/content/drive/MyDrive/BigData_FinalProject/Data/sample.json')
data = json.load(f)
'''

In [ ]:
'''
with open(f'new.json', 'a') as f:
    json.dump(data[0:50000], f)
'''

In [ ]:
#new_data = json.load(open('new.json'))

In [ ]:
#!cp '/content/new.json' '/content/drive/MyDrive/BigData_FinalProject'

### 1. Data Cleaning

# Create Spark Context and Session

In [6]:
# Creating spark context
sc = pyspark.SparkContext(appName="Final_Project")
# Create a spark session
spark = SparkSession(sc)

## Reading the data by Spark (Dataframes)

In [7]:
# Reading the data and combine all parts in a single RDD
path = '/content/drive/MyDrive/BigData_FinalProject/Splits1/*.json'
data_rdd = spark.read.option("multiline","true").json(path).rdd

In [8]:
data_rdd.take(5)

[Row(helpful=['1', '2'], movie='Private Parts (1997)', rating='9', review_date='3 November 1998', review_detail="If you didn't know the man portraying Howard Stern in this movie was himself, you'd think the guy playing the part really nailed it. You never get the sense that Mr. Stern is blowing his own horn:rather, you feel the undercurrent of a genius always on the verge of another breakthrough, and that is the appeal of this film.  Anyone could have made a documentary of Mr. Stern (some have tried, and fell far short of their mark) but no one can deny that Private Parts says it all: it's just a bonus that we see it from the man himself.", review_id='rw0429696', review_summary='Mr. Stern portrays himself with panache...', reviewer='tilly-3', spoiler_tag=0),
 Row(helpful=['29', '41'], movie='Private Parts (1997)', rating=None, review_date='20 July 2001', review_detail='This gem of a flick is the great Rockyesque story of shock jock Howard Stern\'s rise to fame and infamy.  I was thinki

In [9]:
df = data_rdd.toDF()

In [10]:
df.show(5)

+--------+--------------------+------+-----------------+--------------------+---------+--------------------+--------------+-----------+
| helpful|               movie|rating|      review_date|       review_detail|review_id|      review_summary|      reviewer|spoiler_tag|
+--------+--------------------+------+-----------------+--------------------+---------+--------------------+--------------+-----------+
|  [1, 2]|Private Parts (1997)|     9|  3 November 1998|If you didn't kno...|rw0429696|Mr. Stern portray...|       tilly-3|          0|
|[29, 41]|Private Parts (1997)|  null|     20 July 2001|This gem of a fli...|rw0429699|Howard Stern IS T...|       newnoir|          0|
|[31, 37]|Private Parts (1997)|     8|      2 July 2001|I am a big fan of...|rw0429698|Awesome movie! On...|mattymatt4ever|          0|
|  [1, 3]|Private Parts (1997)|  null|22 September 2001|Well, I just watc...|rw0429701|  Got to see that...|     ekokkinis|          0|
|  [6, 8]|Private Parts (1997)|    10|   12 Augu

In [ ]:
#Statistically summarizing about the data
df.describe().show()

In [ ]:
# Check number of rows and columns 
row = df.count()
col = len(df.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the Dataframe is: (50000, 9)
Number of Rows are: 50000
Number of Columns are: 9


In [11]:
#Printing Schema
df.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: long (nullable = true)



In [12]:
df.columns

['helpful',
 'movie',
 'rating',
 'review_date',
 'review_detail',
 'review_id',
 'review_summary',
 'reviewer',
 'spoiler_tag']

In [13]:
from pyspark.sql.types import StringType, DateType, FloatType

df2 = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("spoiler_tag"  ,
              df["spoiler_tag"]
              .cast(FloatType()))

In [14]:
df2.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: float (nullable = true)



In [ ]:
df2.describe().show()

+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|summary|        movie|           rating|    review_date|       review_detail|review_id|     review_summary|          reviewer|       spoiler_tag|
+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|  count|        50000|            45925|          50000|               50000|    50000|              50000|             50000|             50000|
|   mean|         null|6.745998911268372|           null|                null|     null| 289.39285714285717| 791508.3571428572|           0.21898|
| stddev|         null|2.969629967663795|           null|                null|     null|   717.327094646707|223427.99288760958|0.4135591616970949|
|    min|#Alive (2020)|              1.0|1 February 2016|!! Contains a spo...|rw1985329|    !!Great Movie!!|  008_Bob-

### Drop Missing Values

In [15]:
df2.show() #Displaying samples 

+--------+--------------------+------+-----------------+--------------------+---------+--------------------+---------------+-----------+
| helpful|               movie|rating|      review_date|       review_detail|review_id|      review_summary|       reviewer|spoiler_tag|
+--------+--------------------+------+-----------------+--------------------+---------+--------------------+---------------+-----------+
|  [1, 2]|Private Parts (1997)|   9.0|  3 November 1998|If you didn't kno...|rw0429696|Mr. Stern portray...|        tilly-3|        0.0|
|[29, 41]|Private Parts (1997)|  null|     20 July 2001|This gem of a fli...|rw0429699|Howard Stern IS T...|        newnoir|        0.0|
|[31, 37]|Private Parts (1997)|   8.0|      2 July 2001|I am a big fan of...|rw0429698|Awesome movie! On...| mattymatt4ever|        0.0|
|  [1, 3]|Private Parts (1997)|  null|22 September 2001|Well, I just watc...|rw0429701|  Got to see that...|      ekokkinis|        0.0|
|  [6, 8]|Private Parts (1997)|  10.0|   

In [32]:
df_gby = df2.groupby("movie").mean("rating")

In [33]:
df_gby.show()

+--------------------+------------------+
|               movie|       avg(rating)|
+--------------------+------------------+
|Riot (1997 TV Movie)|               7.0|
|I Love You Philli...|  6.03960396039604|
|Super Paper Mario...|               7.0|
|        Birth (2004)| 5.584905660377358|
|       Quills (2000)| 7.066666666666666|
|       Psycho (1960)|  8.95183486238532|
|   The Graves (2009)|1.9285714285714286|
|¿Qué apostamos? (...|              10.0|
|Milf Training Cam...|              null|
|      The Boy (2016)| 6.295833333333333|
|Spring Shower (1932)|               5.0|
|The Walking Dead:...|               6.0|
| Day of Wrath (1943)| 7.454545454545454|
|Arrow: Al Sah-Him...|               9.0|
|CSI: Cyber (2015–...| 4.134831460674158|
|The Curious Case ...| 7.397183098591549|
|   Deadpool 2 (2018)| 6.588397790055248|
|       Parada (2011)| 8.428571428571429|
|Good Vibrations (...|            6.1875|
|Abre los ojos (1997)|               8.2|
+--------------------+------------

In [35]:
df_gby.collect()[1]

Row(movie='I Love You Phillip Morris (2009)', avg(rating)=6.03960396039604)

In [27]:
k = df2.groupBy("movie").sum("rating").show(truncate=False)

+---------------------------------------------------+-----------+
|movie                                              |sum(rating)|
+---------------------------------------------------+-----------+
|Riot (1997 TV Movie)                               |28.0       |
|I Love You Phillip Morris (2009)                   |610.0      |
|Super Paper Mario (2007 Video Game)                |14.0       |
|Birth (2004)                                       |296.0      |
|Quills (2000)                                      |106.0      |
|Psycho (1960)                                      |3903.0     |
|The Graves (2009)                                  |27.0       |
|¿Qué apostamos? (1993–2008)                        |10.0       |
|Milf Training Camp 3 (2013 Video)                  |null       |
|The Boy (2016)                                     |1511.0     |
|Spring Shower (1932)                               |5.0        |
|The Walking Dead: Scars (2019) Season 9, Episode 14|36.0       |
|Day of Wr

In [ ]:
# Replace null values in the ratings column with the computed mean
df = df.na.fill(df_gby, subset=["ratings"])

In [16]:
'''
# Imputer method automatically replaces null values with mean values.
imputer = Imputer(inputCols = ["rating"], outputCols = ["rating-Out"])

#Fitting DataFrame into a model
imputeModel = imputer.fit(df2) 

#Transforming the DataFrame
df3=imputeModel.transform(df2) 
'''

In [19]:
#Describing the dataframe
df3.show()

+--------+--------------------+------+-----------------+--------------------+---------+--------------------+---------------+-----------+----------+
| helpful|               movie|rating|      review_date|       review_detail|review_id|      review_summary|       reviewer|spoiler_tag|rating-Out|
+--------+--------------------+------+-----------------+--------------------+---------+--------------------+---------------+-----------+----------+
|  [1, 2]|Private Parts (1997)|   9.0|  3 November 1998|If you didn't kno...|rw0429696|Mr. Stern portray...|        tilly-3|        0.0|       9.0|
|[29, 41]|Private Parts (1997)|  null|     20 July 2001|This gem of a fli...|rw0429699|Howard Stern IS T...|        newnoir|        0.0|  6.680677|
|[31, 37]|Private Parts (1997)|   8.0|      2 July 2001|I am a big fan of...|rw0429698|Awesome movie! On...| mattymatt4ever|        0.0|       8.0|
|  [1, 3]|Private Parts (1997)|  null|22 September 2001|Well, I just watc...|rw0429701|  Got to see that...|    

In [20]:
#Removing unnecessary columns
df4 = df3.drop(df3['rating'])

In [21]:
#Describing the dataframe
df4.show()

+--------+--------------------+-----------------+--------------------+---------+--------------------+---------------+-----------+----------+
| helpful|               movie|      review_date|       review_detail|review_id|      review_summary|       reviewer|spoiler_tag|rating-Out|
+--------+--------------------+-----------------+--------------------+---------+--------------------+---------------+-----------+----------+
|  [1, 2]|Private Parts (1997)|  3 November 1998|If you didn't kno...|rw0429696|Mr. Stern portray...|        tilly-3|        0.0|       9.0|
|[29, 41]|Private Parts (1997)|     20 July 2001|This gem of a fli...|rw0429699|Howard Stern IS T...|        newnoir|        0.0|  6.680677|
|[31, 37]|Private Parts (1997)|      2 July 2001|I am a big fan of...|rw0429698|Awesome movie! On...| mattymatt4ever|        0.0|       8.0|
|  [1, 3]|Private Parts (1997)|22 September 2001|Well, I just watc...|rw0429701|  Got to see that...|      ekokkinis|        0.0|  6.680677|
|  [6, 8]|Pri

In [ ]:
# Check number of rows and columns 
row = df4.count()
col = len(df4.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the Dataframe is: (50000, 9)
Number of Rows are: 50000
Number of Columns are: 9


# Pipeline

In [ ]:
# Tokenize DataFrame
tokened = Tokenizer(inputCol="review_detail", outputCol="words")
tokened_transformed = tokened.transform(df4)
tokened_transformed.show()

row = tokened_transformed.count()
col = len(tokened_transformed.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+
| helpful|               movie|review_date|       review_detail|review_id|      review_summary|            reviewer|spoiler_tag|rating-Out|               words|
+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+
|  [1, 1]| After Life (2019– )| 3 May 2020|I enjoyed the fir...|rw5704482|Very Strong Season 2|       raeldor-96879|        0.0|       9.0|[i, enjoyed, the,...|
|  [2, 2]|The Valhalla Murd...| 3 May 2020|I know Iceland is...|rw5704483|Icelandic detecti...|             dosleeb|        0.0|       6.0|[i, know, iceland...|
|  [0, 0]|Special OPS (2020– )| 3 May 2020|Except K K , no o...|rw5704484|     Nothing special|     brightconscious|        0.0|       7.0|[except, k, k, ,,...|
|  [5, 9]|   #BlackAF (2020– )| 3 

In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="Wordsfiltered")
removed_frame = remover.transform(tokened_transformed)
removed_frame.show()

row = removed_frame.count()
col = len(removed_frame.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+--------------------+
| helpful|               movie|review_date|       review_detail|review_id|      review_summary|            reviewer|spoiler_tag|rating-Out|               words|       Wordsfiltered|
+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+--------------------+
|  [1, 1]| After Life (2019– )| 3 May 2020|I enjoyed the fir...|rw5704482|Very Strong Season 2|       raeldor-96879|        0.0|       9.0|[i, enjoyed, the,...|[enjoyed, first, ...|
|  [2, 2]|The Valhalla Murd...| 3 May 2020|I know Iceland is...|rw5704483|Icelandic detecti...|             dosleeb|        0.0|       6.0|[i, know, iceland...|[know, iceland, s...|
|  [0, 0]|Special OPS (2020– )| 3 May 2020|Except K K , no o...|rw5704484|     Nothing spe

In [ ]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="Wordsfiltered", outputCol="hashedValues")

# Transform into a DF
hashed_df = hashing.transform(removed_frame)
hashed_df.show()

row = hashed_df.count()
col = len(hashed_df.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+
| helpful|               movie|review_date|       review_detail|review_id|      review_summary|            reviewer|spoiler_tag|rating-Out|               words|       Wordsfiltered|        hashedValues|
+--------+--------------------+-----------+--------------------+---------+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+
|  [1, 1]| After Life (2019– )| 3 May 2020|I enjoyed the fir...|rw5704482|Very Strong Season 2|       raeldor-96879|        0.0|       9.0|[i, enjoyed, the,...|[enjoyed, first, ...|(262144,[2437,125...|
|  [2, 2]|The Valhalla Murd...| 3 May 2020|I know Iceland is...|rw5704483|Icelandic detecti...|             dosleeb|        0.0|       6.0|[i, know, iceland...|[know, iceland, s...|(262144

In [ ]:
df.head(1)

[Row(helpful=['1', '1'], movie='After Life (2019– )', rating='9', review_date='3 May 2020', review_detail="I enjoyed the first season, but I must say I think season 2 is even stronger. Ricky does a great job as both writer, actor and director and brings out the best in a superb supporting cast. If there was one thing I'd change, I'd like to hear him talk about himself less with other people and speak more in the third person, but other than that it's pretty hard to fault this funny yet emotional comedy.", review_id='rw5704482', review_summary='Very Strong Season 2', reviewer='raeldor-96879', spoiler_tag=0)]

In [ ]:
hashed_df.head(1)

[Row(helpful=['1', '1'], movie='After Life (2019– )', review_date='3 May 2020', review_detail="I enjoyed the first season, but I must say I think season 2 is even stronger. Ricky does a great job as both writer, actor and director and brings out the best in a superb supporting cast. If there was one thing I'd change, I'd like to hear him talk about himself less with other people and speak more in the third person, but other than that it's pretty hard to fault this funny yet emotional comedy.", review_id='rw5704482', review_summary='Very Strong Season 2', reviewer='raeldor-96879', spoiler_tag=0.0, rating-Out=9.0, words=['i', 'enjoyed', 'the', 'first', 'season,', 'but', 'i', 'must', 'say', 'i', 'think', 'season', '2', 'is', 'even', 'stronger.', 'ricky', 'does', 'a', 'great', 'job', 'as', 'both', 'writer,', 'actor', 'and', 'director', 'and', 'brings', 'out', 'the', 'best', 'in', 'a', 'superb', 'supporting', 'cast.', 'if', 'there', 'was', 'one', 'thing', "i'd", 'change,', "i'd", 'like', 't

# Training

In [22]:
df_1 = df4['review_detail', 'spoiler_tag']

In [23]:
df_1.show(5)

+--------------------+-----------+
|       review_detail|spoiler_tag|
+--------------------+-----------+
|If you didn't kno...|        0.0|
|This gem of a fli...|        0.0|
|I am a big fan of...|        0.0|
|Well, I just watc...|        0.0|
|I detest Howard S...|        0.0|
+--------------------+-----------+
only showing top 5 rows



In [24]:
# Train test split
training, testing = df_1.randomSplit([0.7, 0.3],1)

In [ ]:
print("Training Dataset Count: " + str(df_1.count()))
print("Training Dataset Count: " + str(training.count()))
print("Test Dataset Count: " + str(testing.count()))

Training Dataset Count: 50000
Training Dataset Count: 34993
Test Dataset Count: 15007


In [25]:
# Show training data 
training.show()

+--------------------+-----------+
|       review_detail|spoiler_tag|
+--------------------+-----------+
|! am reminded of ...|        0.0|
|!!!! SPOILERS !!!...|        1.0|
|!!!SPOILERS!!!Alt...|        1.0|
|!Warning: This re...|        1.0|
|" . . . can keep ...|        1.0|
|" . . . don't the...|        1.0|
|" . . . it's not ...|        1.0|
|" . . . of the Wo...|        1.0|
|" . . . we humans...|        1.0|
|" The scariest Am...|        0.0|
|" Warning Spoiler...|        1.0|
|"21 Jump Street" ...|        0.0|
|"227" is a comedy...|        0.0|
|"50-50 kaus Dur j...|        0.0|
|"8 thousands year...|        1.0|
|"88 Minutes" is a...|        0.0|
|"9" is an animate...|        0.0|
|"A Chump at Oxfor...|        1.0|
|"A Friend of the ...|        0.0|
|"A Gander at Moth...|        1.0|
+--------------------+-----------+
only showing top 20 rows



In [26]:
# Find the data types
training.dtypes


[('review_detail', 'string'), ('spoiler_tag', 'float')]

In [29]:
# LOGISTIC REGRESSION MODEL 

# Create all the steps for the pipeline
label_indexer = StringIndexer(inputCol='spoiler_tag',outputCol='label')
# code to change positive sentiment to 1 values - stringOrderType="frequencyAsc"
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='features')
lr = LogisticRegression(maxIter=20, regParam=0.001)

# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, lr])

# Fit the pipeline to training reviews.
lrmodel = pipeline.fit(training)

# Tranform the model with the testing data
predictions_lr = lrmodel.transform(testing)

predictions_lr.filter(predictions_lr['label'] == 0) \
    .select("review_detail","Wordsfiltered","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

# Evaluate Logistic Regression model
f1_eval = MulticlassClassificationEvaluator(metricName='f1',predictionCol="prediction")
print("Logistic Regression F1 Score: ", f1_eval.evaluate(predictions_lr))
accuracy_score = MulticlassClassificationEvaluator(metricName='accuracy',predictionCol="prediction")
print("Logistic Regression Accuracy: ", accuracy_score.evaluate(predictions_lr))

+------------------------------+------------------------------+------------------------------+-----------+-----+----------+
|                 review_detail|                 Wordsfiltered|                      features|probability|label|prediction|
+------------------------------+------------------------------+------------------------------+-----------+-----+----------+
|A bankrupted aristocrat cal...|[a, bankrupted, aristocrat,...|(262144,[3785,4714,5381,548...|  [1.0,0.0]|  0.0|       0.0|
|Arranca en un Londres que b...|[arranca, en, un, londres, ...|(262144,[183,1350,1493,1823...|  [1.0,0.0]|  0.0|       0.0|
|Season Five opens with this...|[season, five, opens, with,...|(262144,[2061,2977,3276,409...|  [1.0,0.0]|  0.0|       0.0|
|Jagga Jasoos (2017): When a...|[jagga, jasoos, (2017):, wh...|(262144,[2437,3023,3388,392...|  [1.0,0.0]|  0.0|       0.0|
|Aside from space, there can...|[aside, from, space,, there...|(262144,[619,672,896,1277,2...|  [1.0,0.0]|  0.0|       0.0|
|Si pued

In [30]:
### NAIVE BAYES MODEL

# Create all the steps for the pipeline
label_indexer = StringIndexer(inputCol='spoiler_tag',outputCol='label')
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='features')
nb = NaiveBayes(smoothing=1)

# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, nb])

# Fit the pipeline to training reviews.
nbmodel = pipeline.fit(training)

# Tranform the model with the testing data
predictions_nb = nbmodel.transform(testing)

predictions_nb.filter(predictions_nb['label'] == 0) \
    .select("review_detail","Wordsfiltered","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

# Evaluate Naive Bayes model
f1_eval = MulticlassClassificationEvaluator(metricName='f1',predictionCol="prediction")
print("Naive Bayes F1 Score: ", f1_eval.evaluate(predictions_nb))
accuracy_score = MulticlassClassificationEvaluator(metricName='accuracy',predictionCol="prediction")
print("Naive Bayes Accuracy: ", accuracy_score.evaluate(predictions_nb))

+------------------------------+------------------------------+------------------------------+----------------------------+-----+----------+
|                 review_detail|                 Wordsfiltered|                      features|                 probability|label|prediction|
+------------------------------+------------------------------+------------------------------+----------------------------+-----+----------+
|'Tiny Toon Adventures' prov...|['tiny, toon, adventures', ...|(262144,[27,646,1891,2488,2...|[1.0,1.1074082600811119E-16]|  0.0|       0.0|
|Fleischer's Popeye cartoons...|[fleischer's, popeye, carto...|(262144,[3121,3133,6660,702...|[1.0,1.0875790387473111E-16]|  0.0|       0.0|
|''Godhi Banna Sadharna Maik...|[''godhi, banna, sadharna, ...|(262144,[337,1300,1595,3680...|[1.0,1.0868189327780771E-16]|  0.0|       0.0|
|___________________________...|[__________________________...|(262144,[521,2050,2392,2437...|[1.0,1.0845654388358371E-16]|  0.0|       0.0|
|Inglorious B

In [31]:
### SVM MODEL 

# Create all the steps for the pipeline
label_indexer = StringIndexer(inputCol='spoiler_tag',outputCol='label')
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='features')
lsvc = LinearSVC()

# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, lsvc])

# Fit the pipeline to training reviews.
lsvcmodel = pipeline.fit(training)

# Tranform the model with the testing data
predictions_svm = lsvcmodel.transform(testing)

predictions_svm.filter(predictions_svm['label'] == 0) \
    .select("review_detail","Wordsfiltered","features","label","prediction") \
    .show(n = 10, truncate = 30)

# Evaluate Logistic Regression model
f1_eval = MulticlassClassificationEvaluator(metricName='f1',predictionCol="prediction")
print("SVM F1 Score: ", f1_eval.evaluate(predictions_svm))
accuracy_score = MulticlassClassificationEvaluator(metricName='accuracy',predictionCol="prediction")
print("SVM Accuracy: ", accuracy_score.evaluate(predictions_svm))

+------------------------------+------------------------------+------------------------------+-----+----------+
|                 review_detail|                 Wordsfiltered|                      features|label|prediction|
+------------------------------+------------------------------+------------------------------+-----+----------+
|"#Captured" is an amateuris...|["#captured", is, an, amate...|(262144,[1696,2160,4777,538...|  0.0|       0.0|
|"'Odd' Thomas," is the anti...|["'odd', thomas,", is, the,...|(262144,[6346,8538,10049,11...|  0.0|       1.0|
|"'tis a very silly show"Thi...|["'tis, a, very, silly, sho...|(262144,[3924,3928,5381,563...|  0.0|       0.0|
|"...The film is like a batt...|["...the, film, is, like, a...|(262144,[2437,3023,3329,347...|  0.0|       0.0|
|"21 Jump Street" is a comed...|["21, jump, street", is, a,...|(262144,[1968,3613,4106,421...|  0.0|       0.0|
|"A Little Trip To Heaven" i...|["a, little, trip, to, heav...|(262144,[5507,6346,7400,787...|  0.0|    

In [ ]:
# Cross validation for SVM model
from pyspark.ml.feature import HashingTF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, lsvc])

cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=ParamGridBuilder().build(),
                    evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                    numFolds=5)
model_svc = cv.fit(training)
# Tranform the model with the testing data
predictions = model_svc.transform(testing)
predictions.filter(predictions['label'] == 0) \
    .select("review_detail","Wordsfiltered","features","label","prediction") \
    .show(n = 10, truncate = 30)


In [ ]:
# Computing metrics
avgMetricsGrid_svc = model_svc.avgMetrics
print (avgMetricsGrid_svc)

[0.7361204746769922]


In [ ]:
# Cross validation for SVM model
from pyspark.ml.feature import HashingTF
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, nb])

cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=ParamGridBuilder().build(),
                    evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                    numFolds=5)
model_nb = cv.fit(training)
# Tranform the model with the testing data
predictions = model_nb.transform(testing)
predictions.filter(predictions['label'] == 0) \
    .select("review_detail","Wordsfiltered","features","label","prediction") \
    .show(n = 10, truncate = 30)


+------------------------------+------------------------------+------------------------------+-----+----------+
|                 review_detail|                 Wordsfiltered|                      features|label|prediction|
+------------------------------+------------------------------+------------------------------+-----+----------+
|" Good Casting " had indeed...|[", good, casting, ", had, ...|(262144,[6512,7014,7400,978...|  0.0|       0.0|
|" My uncles who were full u...|[", my, uncles, who, were, ...|(262144,[1968,3837,4493,112...|  0.0|       0.0|
|" Noble, my love " was simp...|[", noble,, my, love, ", wa...|(262144,[2325,2977,3924,392...|  0.0|       1.0|
|" Now you see me " is a mov...|[", now, you, see, me, ", i...|(262144,[5454,6512,8538,172...|  0.0|       0.0|
|" Pegasus Market " was a ro...|[", pegasus, market, ", was...|(262144,[2977,6512,8145,142...|  0.0|       0.0|
|"10 seconds" is a British s...|["10, seconds", is, a, brit...|(262144,[4723,8287,12035,17...|  0.0|    

In [ ]:
# Computing metrics
avgMetricsGrid_nb = model_nb.avgMetrics
print (avgMetricsGrid_nb)

# Split Files

In [ ]:
file_list = ['/content/drive/MyDrive/BigData_FinalProject/Data/part-01.json',
             '/content/drive/MyDrive/BigData_FinalProject/Data/part-02.json',
             '/content/drive/MyDrive/BigData_FinalProject/Data/part-03.json',
             '/content/drive/MyDrive/BigData_FinalProject/Data/part-04.json',
             '/content/drive/MyDrive/BigData_FinalProject/Data/part-05.json',
             '/content/drive/MyDrive/BigData_FinalProject/Data/part-06.json']

In [ ]:
import json
k = 0
for file_ in file_list:
  # Open the file to be splitted
  with open((file_), 'r') as f1:
    ll = json.load(f1)
    #total length size of the json file
    print(len(ll))
    #in here 50000 means we getting splits of 50000 reviews
    size_of_the_split=50000
    total = len(ll) // size_of_the_split
    # Number of splits
    print(total+1)
    for i in range(total+1):
      k = k + 1
      json.dump(ll[i * size_of_the_split:(i + 1) * size_of_the_split], 
                open("/content/drive/MyDrive/BigData_FinalProject/Splits/part"+ str(k)+".json", 'w'))
      print(k)



1010293
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1012212
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
1015000
21
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
1019000
21
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
1014997
21
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
499997
10
106
107
108
109
110
111
112
113
114
115
